<a href="https://colab.research.google.com/github/Srinjana/Steganos/blob/main/video_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We install the packages that need to be installed for successful execution.

In [21]:
!pip install stegano

Importing all the necessary packages.

In [22]:
from stegano import lsb
from os.path import isfile, join

In [23]:
# import time, opencv, numpy modules
import cv2
import time  
import numpy as np
import math
import os
import shutil
from subprocess import call, STDOUT


`split_string` is a helper function to split the input secret strings into small portions. As we do not need to hide all the text in the first frame itself, we divide the frames and hide it in many frames of the input video.

In [24]:
def split_string(s_str, count=10):
    per_c = math.ceil(len(s_str)/count)
    c_cout = 0
    out_str = ''
    split_list = []
    for s in s_str:
        out_str += s
        c_cout += 1
        if c_cout == per_c:
            split_list.append(out_str)
            out_str = ''
            c_cout = 0
    if c_cout != 0:
        split_list.append(out_str)
    return split_list

In the function `frame_extraction`, we create a `tmp` directory that stores all the frames we are going to extract from our input video. We define a loop to go through all the frames of the input video and save it in the `tmp` folder with a unique filename that we can easily sort them. Once this is done, we are left with all the frames from the video.

In [25]:
def frame_extraction(video):
    if not os.path.exists("./tmp"):
        os.makedirs("tmp")

    temp_folder = "./tmp"
    print("[INFO] tmp directory has been successfully created.")

    vidcap = cv2.VideoCapture(video)
    count = 0

    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1


In the `encode_string` function we use LSB transform to hide the data into the frames of the video. Now that we have extracted all the frames, we can divide the strings into small chunks and hide each chunk of the message inside a frame using the `lsb.hide()` method. 
We have loaded the input video, pulled it to as many frames as possible, and encoded it. Now we have to put together the frames into a video along with the necessary encryption.

In [26]:
def encode_string(input_string, root="./tmp/"):
    split_string_list = split_string(input_string)
    for i in range(0, len(split_string_list)):
        f_name = "{}{}.png".format(root, i)
        secret_enc = lsb.hide(f_name, split_string_list[i])
        secret_enc.save(f_name)
        print("[INFO] frame {} holds the fragment {}".format(f_name, split_string_list[i]))


In `decode_string` we have to do the exact same steps as with encryption, but in the opposite order. So we extract all the frames from the video and extract the information from the LSB of the individual frames. We then concatenate the fragments to obtain the final decoded string. We use the `lsb.reveal()` method to do so.

In [27]:
def decode_string(video):
    frame_extraction(video)
    secret = []
    root = "./tmp/"
    for i in range(len(os.listdir(root))):
        f_name = "{}{}.png".format(root, i)
        secret_dec = lsb.reveal(f_name)
        if secret_dec == None:
            break
        secret.append(secret_dec)
    
    print("The decoded message is : ")
    print(''.join([i for i in secret]))
    clean_tmp()


We use the `clean_tmp` function below to clean up all the temporary files that were created during execution to prevent conflict if running the code more than once.

In [28]:
def clean_tmp(path="./tmp"):
    if os.path.exists(path):
        shutil.rmtree(path)
        print("[INFO] tmp files have been cleaned up successfully.")


`FFmpeg` is a free and open-source command-line tool for transcoding multimedia files. It contains a set of shared audio and video libraries such as `libavcodec`, `libavformat`, and `libavutil`. With FFmpeg, we can extract audio files from a video, convert our PNG image files into video, and do much more.

To use `FFmpeg` in Python, we have to import call and `STDOUT` from the `subprocess` library.


 ```
 # call(["ffmpeg", "-i","video.mp4" , "-q:a", "0", "-map", "a", "tmp/audio.mp3", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)`
 ``` 


This code will extract the audio from the given video file and save it as “`audio.mp3`” in the `tmp` folder. After we encode the frames with our text, we can then use this audio file to give our encoded video file the proper audio.


```
# call(["ffmpeg", "-i", "tmp/video.mov", "-i", "tmp/audio.mp3", "-codec",
          "copy", "video.mov", "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)
```


This code saves the encrypted video in a file names `video.mov` and reattaches the audio to it. In the end the output video obtained is not of a diminished quality.


In [29]:
def main():
    input_string = input("Enter your Secret Message : ")
    f_name = input("Enter the name of the source video file with extension: ")
    frame_extraction(f_name)
    call(["ffmpeg", "-i", f_name, "-q:a", "0", "-map", "a",
          "tmp/audio.mp3", "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)

    encode_string(input_string)
    call(["ffmpeg", "-i", "tmp/%d.png", "-vcodec", "png", "tmp/video.mov",
          "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)

    call(["ffmpeg", "-i", "tmp/video.mov", "-i", "tmp/audio.mp3", "-codec",
          "copy", "video.mov", "-y"], stdout=open(os.devnull, "w"), stderr=STDOUT)
    clean_tmp()

Driver Code

In [30]:
if __name__ == "__main__":
    while True:
      print("--- SELECT AN OPTION TO PERFORM VIDEO STEGANOGRAPHY ---")
      print("1.Hide a secret message in a video \n 2.Reveal the secret message from the video")
      print("Enter any other key to exit")
      choice = input("Enter your key here: ")
      if choice == '1':
          main()
      elif choice == '2':
          decode_string(input("Enter the name of video file with extension: "))
          print(" ")
      else:
          break


--- SELECT AN OPTION TO PERFORM VIDEO STEGANOGRAPHY ---
1.Hide a secret message in a video 
 2.Reveal the secret message from the video
Enter any other key to exit
Enter your key here: 1
Enter your Secret Message : Hakuna Matata
Enter the name of the source video file with extension: seal.mp4
[INFO] tmp directory has been successfully created.
[INFO] frame ./tmp/0.png holds the fragment Ha
[INFO] frame ./tmp/1.png holds the fragment ku
[INFO] frame ./tmp/2.png holds the fragment na
[INFO] frame ./tmp/3.png holds the fragment  M
[INFO] frame ./tmp/4.png holds the fragment at
[INFO] frame ./tmp/5.png holds the fragment at
[INFO] frame ./tmp/6.png holds the fragment a
[INFO] tmp files have been cleaned up successfully.
--- SELECT AN OPTION TO PERFORM VIDEO STEGANOGRAPHY ---
1.Hide a secret message in a video 
 2.Reveal the secret message from the video
Enter any other key to exit
Enter your key here: 2
Enter the name of video file with extension: video.mov
[INFO] tmp directory has been su